In [2]:
import os, sys; sys.path.append(os.path.abspath('../'))

from inspector.rsa import ESM

In [3]:
# Crawl the data directory
import os
data_dir = r'..\data\baseline'
data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.mat')]

In [ ]:
samples = 2
flattens = 10

In [4]:
# loop over the files
import scipy.io

network_X = []
network_y = []

for f in data_files[:3]:
    data = scipy.io.loadmat(f)
    test_x = data['test_x']
    test_out = data['test_out']

    network_X.append(test_x)
    network_y.append(test_out)

In [5]:
len(network_X), len(network_y)

(3, 3)

In [6]:
network_X[0].shape, network_y[0].shape

((4, 50, 200, 300), (4, 50, 300))

In [7]:
network_X[1].shape, network_y[1].shape

((4, 50, 200, 300), (4, 50, 300))

In [15]:
x_1 = network_X[0]
x_2 = network_X[1]
x_3 = network_X[2]

# F.atten the neurons and timesteps

In [16]:
import numpy as np

# Assuming X1 and X2 are RNN models of shape (4, 50, 200, 300)
# Flatten the time steps and neuron dimensions of each RNN model
X1_flat = np.reshape(x_1, (x_1.shape[0], x_1.shape[1], -1))
X2_flat = np.reshape(x_2, (x_2.shape[0], x_2.shape[1], -1))
X3_flat = np.reshape(x_3, (x_3.shape[0], x_3.shape[1], -1))

In [17]:
# slice only the first samples
X1_flat = X1_flat[:,:samples,:flattens]
X2_flat = X2_flat[:,:samples,:flattens]
X3_flat = X3_flat[:,:samples,:flattens]

In [18]:
print(X1_flat.shape, X2_flat.shape, X3_flat.shape)

(4, 2, 10) (4, 2, 10) (4, 2, 10)


In [19]:
netrep_metric = ESM(verbose = True)
ingestion_report = netrep_metric.ingest([X1_flat, X2_flat, X3_flat], None, preprcessed = True)
energy_distance = netrep_metric.score()

100%|██████████| 3/3 [00:00<00:00, 1495.47it/s]


In [20]:
import seaborn as sns

def plot_distmat(distmat, title = "Distance Metric", out_path = None):
    '''Plot a distance matrix
    New method uses Seaborn to create a heatmap
    '''

    # Set the x and y ticks to the labels
    number_of_networks = len(distmat[0])
    labels = [f"Network {i+1}" for i in range(number_of_networks)]

    # Plot the heatmap using Seaborn
    ax = sns.heatmap(distmat, cmap='coolwarm', annot=True, fmt=".3f",
                     xticklabels=labels, yticklabels=labels, cbar_kws={'label': 'Distance'})
    ax.set_title(title)

    if out_path is not None:
        ax.figure.savefig(out_path)
    else:
        # Show the plot
        ax.show()

In [ ]:
# save the distance matrix
# np.save('./outputs/baseline_flattened_3_nets_2_samples.npy', energy_distance)